In [3]:
import numpy as np
import time
import matplotlib.pyplot as plt
import cv2
import depthai as dai
import os

In [2]:
# Create pipeline
pipeline = dai.Pipeline()
FDIR='./data'
# Define sources and outputs
camRgb = pipeline.create(dai.node.ColorCamera)
monoLeft = pipeline.create(dai.node.MonoCamera)
monoRight = pipeline.create(dai.node.MonoCamera)
ve1 = pipeline.create(dai.node.VideoEncoder)
ve2 = pipeline.create(dai.node.VideoEncoder)
ve3 = pipeline.create(dai.node.VideoEncoder)

ve1Out = pipeline.create(dai.node.XLinkOut)
ve2Out = pipeline.create(dai.node.XLinkOut)
ve3Out = pipeline.create(dai.node.XLinkOut)

ve1Out.setStreamName('ve1Out')
ve2Out.setStreamName('ve2Out')
ve3Out.setStreamName('ve3Out')

# Properties
camRgb.setBoardSocket(dai.CameraBoardSocket.RGB)
monoLeft.setBoardSocket(dai.CameraBoardSocket.LEFT)
monoRight.setBoardSocket(dai.CameraBoardSocket.RIGHT)
# Create encoders, one for each camera, consuming the frames and encoding them using H.264 / H.265 encoding
ve1.setDefaultProfilePreset(30, dai.VideoEncoderProperties.Profile.H264_HIGH)
ve2.setDefaultProfilePreset(30, dai.VideoEncoderProperties.Profile.H264_HIGH)
ve3.setDefaultProfilePreset(30, dai.VideoEncoderProperties.Profile.H264_HIGH)

# Linking
monoLeft.out.link(ve1.input)
camRgb.video.link(ve2.input)
monoRight.out.link(ve3.input)
ve1.bitstream.link(ve1Out.input)
ve2.bitstream.link(ve2Out.input)
ve3.bitstream.link(ve3Out.input)

# Connect to device and start pipeline
with dai.Device(pipeline) as dev:

    # Output queues will be used to get the encoded data from the outputs defined above
    outQ1 = dev.getOutputQueue(name='ve1Out', maxSize=30, blocking=True)
    outQ2 = dev.getOutputQueue(name='ve2Out', maxSize=30, blocking=True)
    outQ3 = dev.getOutputQueue(name='ve3Out', maxSize=30, blocking=True)
    month,day,year=time.localtime().tm_mon,time.localtime().tm_mday,time.localtime().tm_year
    hour,minutes,seconds=time.localtime().tm_hour,time.localtime().tm_min,time.localtime().tm_sec
    dir_name=str(month)+'_'+str(day)+'_'+str(year)+'_'+str(hour)+str(minutes)+str(seconds)
    os.mkdir(FDIR+'/'+dir_name)
    os.mkdir(FDIR+'/'+dir_name+'/videos/')
    os.mkdir(FDIR+'/'+dir_name+'/frames/')
    name=FDIR+'/'+dir_name+'/videos/'+dir_name
    # The .h264 / .h265 files are raw stream files (not playable yet)
    f_mono1=name+'_mono1.h264'
    f_mono2=name+'_mono2.h264'
    f_color=name+'_color.h264'
    with open(f_mono1, 'wb') as fileMono1H264, open(f_color, 'wb') as fileColorH265, open(f_mono2, 'wb') as fileMono2H264:
        print("Press Ctrl+C to stop encoding...")
        while True:
            try:
                # Empty each queue
                while outQ1.has():
                    outQ1.get().getData().tofile(fileMono1H264)

                while outQ2.has():
                    outQ2.get().getData().tofile(fileColorH265)

                while outQ3.has():
                    outQ3.get().getData().tofile(fileMono2H264)
            except KeyboardInterrupt:
                # Keyboard interrupt (Ctrl + C) detected
                break
    cmd = "ffmpeg -framerate 30 -i {} -c copy {}"
    
    os.system(cmd.format(f_mono1,name+'_mono1.mp4'))
    os.system(cmd.format(f_mono2, name+'_mono2.mp4'))
    os.system(cmd.format(f_color, name+"_color.mp4"))
    time.sleep(5)
#     if(mono1_mp4 in list_dir and mono2_mp4 in list_dir and color_mp4 in list_dir)
#     while True:
#         list_dir=os.list_dir(FDIR+'/'+dir_name)
#         mono1_mp4=name+'_mono1.mp4'
#         mono2_mp4=name+'_mono2.mp4'
#         color_mp4=name+'-color.mp4'
        
    os.remove(f_mono1)
    os.remove(f_mono2)
    os.remove(f_color)

Press Ctrl+C to stop encoding...


In [1]:
import json

In [2]:
json.load

In [13]:
with open('config.json','rb') as file:
    x=json.load(file)

In [14]:
x

{'record': {'data_path': './data',
  'video_encoder_prop': 'H264_HIGH',
  'frame_rate': 30,
  'color_cam': True,
  'mono1': True,
  'mono2': True},
 'frame_extraction': {}}

In [16]:
x['record']

{'data_path': './data',
 'video_encoder_prop': 'H264_HIGH',
 'frame_rate': 30,
 'color_cam': True,
 'mono1': True,
 'mono2': True}

In [10]:
ve1.setDefaultProfilePreset?

In [12]:
dev.getOutputQueue?

In [ ]:
dai.Device(pipeline).getOutputQueue

In [4]:
dai.Device

depthai.Device

In [10]:
# 14442C10913365D300 XLinkDeviceState.X_LINK_UNBOOTED Gehan
# 14442C10810665D300 XLinkDeviceState.X_LINK_UNBOOTED XAvi


for device in dai.Device.getAllAvailableDevices():
    print(f"{device.getMxId()} {device.state}")

14442C10913365D300 XLinkDeviceState.X_LINK_UNBOOTED
14442C10810665D300 XLinkDeviceState.X_LINK_UNBOOTED


In [11]:
pipeline

In [16]:
dai.Device.getAllAvailableDevices()

[<depthai.DeviceInfo at 0x2c5ad6aa170>, <depthai.DeviceInfo at 0x2c5ad72b6b0>]

In [14]:
with dai.Device(pipeline) as dev:
    print()

RuntimeError: Failed to find device after booting, error message: X_LINK_DEVICE_NOT_FOUND

In [15]:
dai.Device()

RuntimeError: Failed to find device after booting, error message: X_LINK_DEVICE_NOT_FOUND

In [21]:
with dai.Device.getDeviceByMxId("14442C10913365D300")(pipeline) as dev:
    print(dev)

TypeError: 'tuple' object is not callable